In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai


In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

### Setup: Round robbin panel of night city representatives discussing Dogtown's independence.

In [ ]:
gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.5-flash"

gpt_system = "You are a chatbot version of a Cyberpunk 2077 StreetKid. Taking part in a round robbin panel comprised of a Corpo and a Nomad, discussing the impact of dogtown's current 'independence'. You are first in the panel. Try not to repeat yourself and respond to comments from each panel participant."

claude_system = "You are a chatbot version of Cyberpunk 2077 Corpo. Taking part in a round robbin panel, comprised of a Streetkid and a Nomad, discussing the impact of dogtown's current 'independence'. You are second in the panel. Try not to repeat yourself and respond to comments from each panel participant."

gemini_system = "You are a chatbot verion of Cyberpunk 2077 Nomad. Taking part in a round robbin panel, comprised of a Corpo and a Streetkid, discussing the impact of dogtown's current 'independence'. You are last in the panel. Try not to repeat yourself and respond to comments from each panel participant."

gemini_messages = ["Thanks for having me."]
gpt_messages = ["Howdy!"]
claude_messages = ["Hi."]

In [ ]:
def call_gpt(): #Streetkid
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "user", "content": gemini})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

def call_claude(): #Corpo
    messages = []
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=750
    )
    return message.content[0].text


def call_gemini(): #Nomad
    messages = [{"role": "system", "content": gemini_system}]
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": claude_message})
        messages.append({"role": "assistant", "content": gemini})
    messages.append({"role": "user", "content": claude_messages[-1]})
    gemini_via_openai_client = OpenAI(
        api_key=google_api_key, 
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    response = gemini_via_openai_client.chat.completions.create(
        model="gemini-2.5-flash",
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
print(f"Streetkid:\n{gpt_messages[0]}\n")
print(f"Corpo:\n{claude_messages[0]}\n")
print(f"Nomad:\n{gemini_messages[0]}\n")

for i in range(3):
    gpt_next = call_gpt()
    print(f"Streetkid:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Corpo:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Nomad:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)